# Scraping Horse Racing Odds

In [ ]:
from selenium import webdriver
from bs4 import BeautifulSoup
import pandas as pd
import re, os.path

In [ ]:
browser = webdriver.Firefox(executable_path=os.path.expanduser('~/.local/bin/geckodriver'))

In [ ]:
browser.get('https://sport.netbet.com/horse-racing/')

## Select Course and Time

Find the `<img>` tags which are the toggles for each of the races.

The tags that we are looking for look something like this:

```
<img src="/i/none_v.gif" onclick="HorseRacingBranchWindow.toggleSelection(33668677); cancelBubble(event);">
```

In [ ]:
toggles = browser.find_elements_by_css_selector('img[src="/i/none_v.gif"]')

Retrieve the script for the first toggle.

In [ ]:
toggle_script = toggles[0].get_attribute("onclick")
toggle_script

We actually only want the first statement.

In [ ]:
toggle_script = toggle_script.split(' ')[0]

In [ ]:
browser.execute_script(toggle_script)

Locate the *View* button and give it a click.

In [ ]:
view_button = browser.find_element_by_link_text('View')

In [ ]:
view_button.click()

To get the data for all of the race we could select multiple races at once or handle each of them independently. The latter approach might be somewhat simpler to implement but would place a heavier load on the server due to multiple requests.

**Note:** In the code above we have used Selenium to parse the page. This works fine as long as the statements are executed in quick succession. However the page that we are looking at refreshes periodically, so it's also possible that the element references can become stale. So it might make more sense to transfer the parsing process to Beautiful Soup, which operates on a static copy of the page.

## Retrieve Race Details

Since the page content has changed we'll need to make fresh soup.

You'll need to wait for the page to be fully rendered in the browser window before running this. In production you'd need to check that the page was loaded.

In [ ]:
soup = BeautifulSoup(browser.page_source, 'lxml')

Find container with the details of the race.

In [ ]:
event = soup.find(attrs = {'class': 'horseEvent__container'})
event

In [ ]:
title = event.find(attrs = {'class': 'horseEvent__title'}).text
title

That's two separate pieces of information in a single string. We'll need to parse those out.

In [ ]:
title = re.match('([^\d]*)(\d\d:\d\d) *', title)
#
if title:
    course = title.group(1).strip()
    time = title.group(2)
#
del(title)

In [ ]:
race = event.find(attrs = {'class': 'horseRacing-eventInfo-eventName'}).text.strip()

In [ ]:
print('"%s": %s at %s' % (race, time, course))

## Retrieve Horses and Odds

Locate the required table.

In [ ]:
odds_table = soup.find('table', attrs = {'class': 'oddsTable'})

Loop through rows in table, skipping over first row which contains headers and rows for favourites. Simple exception handling allows this to be done quite easily because the required tags are missing for those rows.

In [ ]:
rows = []

for tr in odds_table.find_all('tr'):
    try:
        stall  = tr.find(attrs = {'class': 'horseRacing-number-horse'}).text
        horse  = tr.find(attrs = {'class': 'horseRacing-horseName'}).text
        jockey = tr.find(attrs = {'class': 'horseRacing-jockeyName'}).text
        weight = tr.find(attrs = {'class': 'horseRacing-horseWeight'}).text
        age    = tr.find(attrs = {'class': 'horseRacing-horseAge'}).text
        odds   = tr.find(attrs = {'class': 'racing-tableColumn-odds'}).text
        # Split trainer and jockey names.
        trainer, jockey = [name.strip() for name in jockey.split(' / ')]
        # Clean up fields with cruft.
        horse = horse.strip()
        age = age.replace('Age - ', '').strip()
        # Convert odds and age to numeric.
        odds = float(odds)
        age = int(age)
        rows.append({
            'stall': stall,
            'horse': horse,
            'jockey': jockey,
            'trainer': trainer,
            'weight': weight,
            'age': age,
            'odds': odds,
        })
    except AttributeError:
        pass

In [ ]:
rows

Now convert the list into a data frame.

In [ ]:
df = pd.DataFrame(rows)
del rows

In [ ]:
df

## Possible Extensions

Here are some ideas for extensions:

1. The odds field actually has a script attached to it which places a bet when clicked. We could capture the details of that script for automated betting.

## Clean Up

In [ ]:
browser.close()